In [1]:
import numpy as np
import pandas as pd
import time
from functools import partial

from scipy.stats import randint, uniform
from sim_lib import simulation

pd.options.mode.chained_assignment = None


#### Eps-greedy (baseline)

In [3]:
def eps_greedy(history: pd.DataFrame, eps: float):
    if uniform.rvs() < eps:
        n = history.shape[0]
        return history.index[randint.rvs(0, n)]

    ctr = history['clicks'] / (history['impressions'] + 10)
    n = np.argmax(ctr)
    return history.index[n]

policy = partial(eps_greedy, eps=0.06)

In [4]:
# seed for homework
seed = 18475
np.random.seed(seed=seed)

start = time.time()
output = simulation(policy, n=200000, seed=seed)
end = time.time()
end - start

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated


682.381539106369

In [5]:
# baseline regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(1540.7609683932544, 0.007703804841966272, 184)

In [6]:
output['history']

,impressions,clicks,lifetime,p
153,18970.0,4199.0,18003.025431,0.220134
162,228.0,26.0,1537.166719,0.113780
172,163.0,32.0,19648.592394,0.219968
173,170.0,18.0,12771.474990,0.122694
180,26.0,3.0,4655.819793,0.020061
182,6.0,0.0,889.624649,0.004621
183,1.0,0.0,15187.163761,0.073886


#### UCB

In [7]:
class ucb:
    def __init__(self, c: float = 0.1):
        self.c = c
    
    def __call__(self, history: pd.DataFrame):
        # номер шага - сколько всего было показов
        t = history['impressions'].sum() + 1
        # оценка на матожидание награды
        theta = history['clicks'] / (history['impressions'] + 1)
        # добавка в UCB для exploration
        correction = np.sqrt(np.log(t) / (history['impressions'] + 1))
        # выбираем действие - какую ручку дергать
        action = np.argmax(theta + self.c * correction)
        return history.index[action]

Подберем параметры

In [8]:
seed = 18475
cs = [0.0001, 0.001, 0.01, 0.1, 1]
regrets = []

for c in cs:
    print(f'--- c = {c} ---')
    np.random.seed(seed=seed)
    policy = ucb(c=c)
    start = time.time()
    output = simulation(policy, n=200000, seed=seed)
    end = time.time()
    regrets.append([c, output['regret']])
    print(f"regret = {output['regret']:.3f}, mean_regret: {output['regret']/output['rounds']:.4f}")
    print(f'time: {end - start} s', '\n')

--- c = 0.0001 ---
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
regret = 7570.250, mean_regret: 0.0379
time: 669.4210300445557 s 

--- c = 0.001 ---
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions ha

In [9]:
regrets = np.array(regrets)
pd.DataFrame(regrets, columns=['c', 'regret'])

,c,regret
0,0.0001,7570.250349
1,0.0010,7570.250349
2,0.0100,7791.756275
3,0.1000,189.570644
4,1.0000,6428.429018


Видно, что при `c=0.1` получается наименьший `regret=189.6`, что сильно лучше, чем у бейзлайна: `regret=1540`